In [1]:
import ollama
import pandas as pd

## Classification Schema Selection

Selection:

binary1: irrelevant vs. explicit/implicit/relchanges/releconomy

binary2: relchanges/releconomy/irrelevant vs. explicit/implicit

binary3: releconomy/irrelevant vs. explicit/implicit/relchanges

three_code1: irrelevant vs. implicit/relchanges/releconomy vs. explicit

three_code2: irrelevant vs. relchanges/releconomy vs. explicit/implicit

three_code3: releconomy/irrelevant vs. relchanges vs. explicit/implicit

four_code: irrelevant vs. relchanges/releconomy	 vs. explicit vs. implicit

four_code2: irrelevant vs. relchanges vs. explicit/implicit vs. releconomy

five_code: irrelevant vs. relchanges vs. explicit vs. releconomy vs. implicit
    

In [ ]:
corpus = pd.read_json('../data/old_data_clean.json')

output_list = []

meta_statement = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

prompt = "Is the article relevant to American economic inequality? Answer relevant or irrelevant, and why in 1 sentence."

for num in range(len(corpus)):
    text = 'Title: ' + corpus.loc[num,'title'] + '\n' + corpus.loc[num,'text']
    print(num)
    print(text[:20])

    response = ollama.chat(
        model="llama3.1:70b",
        messages=[
            {
                "role": "system",
                "content": "You are a news classifier."
            },
            {
                "role": "user",
                "content": meta_statement,
            },

            {
                "role": "user",
                "content": "Read this article: " + text,
            },
            {
                "role": "user",
                "content": prompt

            },
        ],
    options={
    "seed": 101,
    "temperature": 0,
    "num_ctx": 128200,
    "num_thread": 30,
    #"num_predict": 10,
      }
    )
    output_list.append((response["message"]["content"]))

In [ ]:
corpus['content'] = output_list

corpus['gen_code'] = corpus['content']

corpus.loc[corpus['content'].str.lower().str.startswith('relevant'), 'gen_code'] = 1
corpus.loc[corpus['content'].str.lower().str.startswith('irrelevant'), 'gen_code'] = 0

columns = ['title', 'year', 'month', 'journal', 'code', 'weight', 'code_label', 'id', 'code', 'content', 'gen_code']
corpus = corpus[columns]

In [24]:
corpus.to_csv('../data/inequality_dataset_llama3_1-70b-NoDefinition.csv',